# Notebook: F5 -- Generalization performance
Authors: Hugo Toll, Paul Häusner<br>
Date: Nov 2023
Updated: Jan 2024

This notebook is complementary to lecture F5 about assessing the generalization performance of machine learning models in order to highlight the key concepts. The focus will be on

- How to estimate expected error on new data $E_{new}$ for a given model
- How to both train a model and estimate its expected error based on data

Please read the instructions and play around with the notebook where it is described.

In [ ]:
# import the neccessary libraries
import numpy as np
import matplotlib.pyplot as plt

---

## 1. Creating data and train a regression model on it

When applying machine learning techniques we usually assume that there exists a joint distribution of the inputs and outputs $p(x, y)$. In practice, we have, however, no access to this distribution but can only obtain samples from it.

For the sake of this notebook, we assume that we know the true joint probability distribution of the inputs and the outputs which is given by a joint normal distribution

$$ [x, y]^T \sim \mathcal{N}(\mu, \Sigma) $$

where $\mu$ is the mean of the distribution and $\Sigma$ is the covariance matrix.

To make things a little more concrete, assume that $x$ stands for the square meters of a house and $y$ stands for the selling price of it (in thousands)

The following code specifies the just explained distribution and allows us to sample points from it.

In [ ]:
mu = np.array([100, 200])
Sigma = np.array([[100, 50], [50, 100]])

def sample():
    s = np.random.multivariate_normal(mu, Sigma)
    return s[0], s[1]

def dataset(n, seed=None):
    if seed is not None:
        np.random.seed(seed)
    d = [sample() for i in range(n)]
    x_d = [s[0] for s in d]
    y_d = [s[1] for s in d]
    return np.array(x_d), np.array(y_d)

Now, lets sample $n=15$ data points from our created joint distribution and plot the each point in the graph.

In [ ]:
x, y = dataset(15, seed=42)

# plotting some samples from the distribution
plt.plot(x, y, "x", ms=7)
plt.grid(alpha=.3)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Our aim is to find a linear regression model with two parameters (offset and slope) which we usually denote by $\theta \in \mathbb{R}^2$ and evaluate how good it performs when seeing new samples from the joint probability distribution.

Question:
- Look at the samples from the distribution above. What could be potential values for the linear regression coefficients?

---

## 2. Evaluating a fixed model

Let's assume that a friend of ours is an expert in the domain and tells us that the parameters for the regression problem should be $\theta = [90, 1.5 ]^T $. We want to check now if these parameters are a good choice.

Therefore, we want to compute the expected error the model makes when seeing new samples from the joint distribution. This error is called $E_{new}$ and is given by

$$E_{new}(\theta) = \mathbb{E}_\star \left[ E(x_\star, y_\star; \theta) \right].$$

Here, $E(x,y)$ is an error function that compares a prediction $\hat{𝑦}(x)$ to the true value $𝑦$. It returns a small value if $\hat{𝑦}(x)$ is a good prediction of $𝑦$. However, we can not compute the expected value in the equation above analytically. Instead, we have to solve the problem by estimating $E_{new}$ using samples from the data distribution in the form of a set of independent and identically distributed (iid) samples $\{ (x_i, y_i) \}_{i=1}^n$ which approximates the expected value

$$E_{new}(\theta) \approx \frac1n \sum_{i=1}^n E(x_i, y_i; \theta).$$

In [ ]:
theta = np.array([90, 1.5]) # given model parameters

In [ ]:
# compute the error of the prediciton
# function E(x, y, \theta)
def compute_error(theta, x, y):
    model_prediction = (theta[0] + theta[1] * x)
    return np.mean(np.abs(model_prediction - y))

In [ ]:
# function to compute and plot E_new
def compute_e_new(n, theta):
    np.random.seed(553)

    errors = []
    for i in range(n):
        x_star, y_star = sample()
        error_i = compute_error(theta, x_star, y_star)
        errors.append(error_i)

    # compute estimation after n steps:
    E_new = [np.mean(errors[:(i+1)]) for i in range(n)]
    print(f"Estimated $E_new$ after {n} steps: {E_new[-1]}")

    # Plot the estimated new error after n steps
    plt.plot(range(1, n+1), E_new, ".-", label="$E_{new}$")
    plt.grid(alpha=.3)
    plt.xlabel("Number of samples: $n$")
    plt.legend()
    plt.show()

In [ ]:
# task: change n
n = 3

compute_e_new(n, theta)

Questions:
- What happens if you change $n$ in the code above? What is the ideal value of $n$?
- What is the drawback of large $n$ values? What is the drawback of small $n$ values?
- Which error metric is used in the example above?
- Go to the code above and increase $n$. Do the results match with your answer to the first question?

---

## 3. Evaluating a learned model

We now additionally want to train the linear regression model on data we sample instead of using a fixed set of model weights provided to us. This is achieved by solving the normal equations. For details on this please refer to the material on linear regression from the previous lectures.

In [ ]:
# train a linear regression model  given data x and y
def fit_lr(x, y):
    x_augmented = np.vstack((np.ones_like(x), x)).T
    theta = np.linalg.solve(x_augmented.T@x_augmented, x_augmented.T@y)
    return theta

In [ ]:
# create data to train model on
x, y = dataset(15, seed=42)

# find model parameters theta_t based on dataset
theta_t = fit_lr(x, y)

print(f"Learned model parameters $theta(T) = {theta_t}$ ")

# plot the data and the linear regression model
plt.plot(x, y, "x", ms=7, label="data")
plt.plot(np.linspace(np.min(x), np.max(x)), theta_t[0] + theta_t[1] * np.linspace(np.min(x), np.max(x)),
         label="linear regression")
plt.grid(alpha=.3)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

However, one problem is that when we train a model on data $\mathcal{T}$, the parameters of that model -- which are here denoted by $\theta(\mathcal{T})$ to indicate that they are obtained from training on the samples $\mathcal{T}$ -- become dependent on the dataset used to train them i.e.

$$E_{new}(\theta(\mathcal{T})) = \mathbb{E}_\star \left[ E(x_\star, y_\star; \theta(\mathcal{T})) \right].$$

This means we can not estimate $E_{new}$ with the sampled dataset $\mathcal{T}$ we used to train the model, since the samples are directly influencing the model weights. When evaluating the model with parameters $\theta(\mathcal{T})$, on the dataset $\mathcal{T}$, we obtain the training error denoted $E_{train}$ which is an underapproximation of $E_{new}$.

Question:
- How are the model parameters in the learned case different from the parameters used in Section 2? Which model do you expect to perform better in terms of E_new?

In [ ]:
train_error = compute_error(theta_t, x, y)
print(f"The training error of the trained LR model is {train_error}")

Similar, to the previous task, to estimate $E_{new}$ we need to sample new points from the distribution $p(x, y)$ which are used to numerically approximate the expected value as before. Since we have access to the data distribution we can easily achieve this.

In [ ]:
# in order to produce a good estimate of E_new, we use 1000 samples here.
def test(theta, n=1000):
    x_test, y_test = dataset(n, seed=42**2)
    test_error = compute_error(theta, x_test, y_test)
    return test_error

print(f"The expected new error of the trained LR model is approximate {test(theta_t)}")

We can see that the test error $E_{new}$ is usually higher than the training error $E_{train}$. We call the difference between them the **generalization error**.

Now, we want to observe what happens with $E_{new}$ and $E_{train}$ when we increase the number of samples we are training the model parameters on.

In [ ]:
# function to compute and plot E_train and E_new
def compute_e_train_e_new(n):
    e_new = []
    e_train = []

    for i in range(2, n+1):
        np.random.seed(42)

        x_i, y_i = dataset(i)
        theta_i = fit_lr(x_i, y_i)
        e_train.append(compute_error(theta_i, x_i, y_i))
        e_new.append(test(theta_i))

    plt.plot(range(2, n+1), e_train, ".-", label="E_train")
    plt.plot(range(2, n+1), e_new, ".-", label="E_new")
    plt.grid(alpha=0.3)
    plt.ylabel("")
    plt.xlabel("Number of samples: $n$")
    plt.legend()
    plt.show()

In [ ]:
n = 3 # change me!!
compute_e_train_e_new(n)

Questions:
- What is $E_{train}$ for $n=2$ and why?
- What happens when we change the number of samples when estimating the parameters of the model?
- What happens if you rerun the code? Do you get the same results? Why / why not?

Rerun the code above with a different choice for $n$ and see if the results match with the answers to your questions.

In practice we usually only have one dataset and it is usually not trivial to obtain new samples. Therefore, to get an estimate of $E_{new}$, we need to split our dataset $\mathcal{T}$ into two parts: the training data and test data. We use the training data $\mathcal{T}_{train}$ to train the linear regression model and use $\mathcal{T}_{hold-out}$ to estimate $E_{new}$. This is also called **hold-out validation**.

---

## Take home messages:

- We can not compute the expected new error $E_{new}$ of a model analytically and therefore need to approximate it using data.
- When we both train a model on data and evaluate its expected error on new samples, we need to make sure that the evaluation is independent of the model.
- The generalization gap is the difference between the error the model makes on the training data and the error the model makes on new, unseen data.

**Warning**: When we are choosing the hyperparameters (such as regularization strength, hand-engineered features, etc.) of a learned model based on the hold-out data, we also influence the model weights using the data which we are using to estimate the expected new error.
Therefore, usually two seperate test sets are used: the first one is also called validation set and is used to select which model to use. The second one is used to estimate the performance of the chosen model. It is important that no model selection is made based on this final number.

**Recommended reading**:
- Machine Learning - A First Course for Engineers and Scientists, Chapter 4